# Circulation newtwork

A Rapido request contains a lending request and a corresponding borrowing request. The reports are the result of merging the borrowing requests information into the lending requests information.

Lending requests are master.

The lending requests are splitted into 3 parts. Otherwise it could be hard to download data at once.

This script  uses 5 csv files to build the reports:
* NZ_RAPIDO_borrowing.csv: list of borrowing requests
* NZ_RAPIDO_borrowing_Renouvaud.csv: borrowing requests of Renouvaud. There is no lending request if Renouvaud fulfill the request. Renouvaud is however as partner in borrowing request.
* NZ_RAPIDO_lending.csv: main information of lending requests
* NZ_RAPIDO_lending_bib.csv: bibliographical data of lending requests
* NZ_RAPIDO_lending_part.csv: partner info of lending requests

In analytics, two reports need to be updated with date information:
* Lending_requests_external_ids
* NZ_RAPIDO_borrowing_Renouvaud

Reports are NZ Analytics: `/Shared Folders/SLSP Network Zone 41SLSP_NETWORK/Reports/SLSP_courier_statistics`

Documentation on SLSPhere: https://slsp.atlassian.net/wiki/spaces/PSI/pages/465404065/Network+circulation+statistics



In [1]:
# Import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
from datetime import date
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font
from copy import copy
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

In [2]:
# Tools used to build and design Excel sheet
def copy_cell_contents(source_sheet, target_sheet):
    """Copy one to one content and format of a cell
    """
    mr = source_sheet.max_row
    mc = source_sheet.max_column
    # copying the cell values from source 
    # excel file to destination excel file

    for i in range (1, mr + 1):
        for j in range (1, mc + 1):
            # reading cell value from source excel file
            source_cell = ws_exp.cell(row = i, column = j)

            # writing the read value to destination excel file
            target_cell = target_sheet.cell(row = i, column = j)
            target_cell.value = source_cell.value

            if source_cell.has_style:
                target_cell.font = copy(source_cell.font)
                target_cell.border = copy(source_cell.border)
                target_cell.fill = copy(source_cell.fill)
                target_cell.number_format = copy(source_cell.number_format)
                target_cell.protection = copy(source_cell.protection)
                target_cell.alignment = copy(source_cell.alignment)

def copy_sheet_attributes(source_sheet, target_sheet):
    """Copy format of an entire sheet.
    """
    # set row dimensions
    # So you cannot copy the row_dimensions attribute. Does not work (because of meta data in the attribute I think). So we copy every row's row_dimensions. That seems to work.
    for rn in range(len(source_sheet.row_dimensions)):
        target_sheet.row_dimensions[rn] = copy(source_sheet.row_dimensions[rn])

    if source_sheet.sheet_format.defaultColWidth is None:
        pass
    else:
        target_sheet.sheet_format.defaultColWidth = copy(source_sheet.sheet_format.defaultColWidth)

    # set specific column width and hidden property
    # we cannot copy the entire column_dimensions attribute so we copy selected attributes
    for key, value in source_sheet.column_dimensions.items():
        target_sheet.column_dimensions[key].min = copy(source_sheet.column_dimensions[key].min)   # Excel actually groups multiple columns under 1 key. Use the min max attribute to also group the columns in the targetSheet
        target_sheet.column_dimensions[key].max = copy(source_sheet.column_dimensions[key].max)  # https://stackoverflow.com/questions/36417278/openpyxl-can-not-read-consecutive-hidden-columns discussed the issue. Note that this is also the case for the width, not onl;y the hidden property
        target_sheet.column_dimensions[key].width = copy(source_sheet.column_dimensions[key].width) # set width for every column
        target_sheet.column_dimensions[key].hidden = copy(source_sheet.column_dimensions[key].hidden)

def set_columns_width(ws):
    """Used to define the width of the various columns of the report.
    """
    columns_width = {'A': 25,
 'B': 22,
 'C': 20,
 'D': 20,
 'E': 12,
 'F': 12,
 'G': 18,
 'H': 12,
 'I': 12,
 'J': 18,
 'K': 12,
 'L': 18,
 'M': 18,
 'N': 18,
 'O': 18,
 'P': 15,
 'Q': 25,
 'R': 10,
 'S': 10,
 'T': 10,
 'U': 10,
 'V': 10,
 'W': 18,
 'X': 18,
 'Y': 20,
 'Z': 20,
 'AA': 20,
 'AB': 12,
 'AC': 8,
 'AD': 8,
 'AE': 10,
 'AF': 10}
    for e in columns_width:
        columns_width[e] *= 1.1

    dim_holder = DimensionHolder(worksheet=ws)

    for col in range(ws.min_column, ws.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=columns_width[get_column_letter(col)])
    ws.column_dimensions = dim_holder

def transform_iz_code(raw_code):
    """Harmonize IZ codes
    """
    
    if raw_code == '41BIG_INST':
        return 'BVERW'
    
    inst = raw_code.split('_')[1]
    if inst == 'FHO':
        inst = 'OSTGR'
    if inst == 'SUP':
        inst = 'SUPSI'
    if inst == 'PHZ':
        inst = 'ZPH'
    if inst == 'RRO':
        inst = 'ISR'
    if inst == 'UGE':
        inst = 'UNIGE'
    if inst == 'UNE':
        inst = 'UNINE'
    if inst == 'FNW':
        inst = 'FHNW'
    if inst == 'ZHK':
        inst = 'ZHDK'
    if inst == 'HES':
        inst = 'HESSO'
    if inst == 'EPF':
        inst = 'EPFL'
    if inst == 'IID':
        inst = 'IHEID'
    if inst == 'LIB':
        inst = 'LIB4RI'
    if inst == 'ZAW':
        inst = 'ZHAW'
    if inst == 'RBE':
        inst = 'BSB'
    return inst

In [3]:
# Load data
lend_req_base = pd.read_csv('data/NZ_RAPIDO_lending.csv', dtype=str)
lend_req_bib = pd.read_csv('data/NZ_RAPIDO_lending_bib.csv', dtype=str)
lend_req_part = pd.read_csv('data/NZ_RAPIDO_lending_part.csv', dtype=str)
bor_req = pd.read_csv('data/NZ_RAPIDO_borrowing.csv', dtype=str)
bor_to_ren_req = pd.read_csv('data/NZ_RAPIDO_borrowing_Renouvaud.csv', dtype=str)

In [4]:
# Merge lending requests data: base + partners
lend_req = lend_req_base.merge(lend_req_bib, on='External Request Id', how='left').merge(lend_req_part, on='External Request Id', how='left')

In [5]:
# Cleaning, lending requests

# If not physical item is sent, the row is deleted
lend_req = lend_req.loc[lend_req['Lending Request Status']!='Shipped Digitally']

# Without external id, it is impossible to match with borrowing requests
lend_req = lend_req.loc[~pd.isnull(lend_req['External Request Id'])]

# Note as test request containing "Test"
lend_req['Lending Is test'] = lend_req['Note'].str.contains('Test', case=False)

# Harmonize Institution codes
lend_req['Institution Code'] = lend_req['Institution Code'].apply(transform_iz_code)

# Create a new column with the external id without the prefix. It is used to match with the borrowing requests
lend_req['Simple external Id'] = lend_req['External Request Id'].apply(lambda x: x.split('//')[-1])

In [6]:
# Cleaning, borrowing requests

# Note the requests as test if a note contains "Test" 
bor_req['Borrowing Is test'] = bor_req['Note'].str.contains('Test', case=False)
bor_to_ren_req['Borrowing Is test'] = bor_to_ren_req['Note'].str.contains('Test', case=False)

# Harmonize Institution codes
bor_req['Institution Code'] = bor_req['Institution Code'].apply(transform_iz_code)
bor_to_ren_req['Institution Code'] = bor_to_ren_req['Institution Code'].apply(transform_iz_code)

In [7]:
lend_req.columns

Index(['External Request Id', 'Internal Id', 'Item Sent Date (Calendar)',
       'Lending Request Status', 'Note', 'Institution Code', 'Library Code',
       'Library Name', 'Permanent Physical Location Code',
       'Permanent Call Number', 'Barcode', 'MMS Id', 'Network Id', 'ISSN',
       'ISBN', 'Title', 'Partner Id', 'Partner Name', 'Lending Is test',
       'Simple external Id'],
      dtype='object')

In [8]:
# Rename columns
lend_req.columns = ['External Request Id', 'Lending Internal Id', 'Item Sent Date',
       'Lending Request Status', 'Lending Note', 'From IZ', 'From Library Code',
       'From Library Name', 'Permanent Physical Location Code',
       'Permanent Call Number', 'Barcode', 'MMS Id', 'Network Id', 'ISSN', 'ISBN',
       'Title', 'Borrowing Partner Id', 'Borrowing Partner Name', 'Lending Is test', 'Simple external Id']

In [9]:
bor_req.columns

Index(['External Identifier', 'Internal Id', 'Shipping Date (Calendar)',
       'Item Loan Date (Calendar)', 'Item Return Date (Calendar)',
       'Item Arrival Date (Calendar)', 'Lending Institution',
       'Cancellation Reason', 'Borrowing Request Status',
       'Request Completed Indicator', 'Is Filled', 'Note', 'User Group Code',
       'Institution Code', 'Library Code', 'Library Name',
       'Borrowing Creation Date', 'Partner Id', 'Partner Code', 'Request Type',
       'Borrowing Is test'],
      dtype='object')

In [10]:
# Rename columns
bor_req.columns = ['External Identifier', 'Borrowing Internal Id',
       'Shipping Date', 'Item Loan Date',
       'Item Return Date',
       'Item Arrival Date', 'Lending Institution',
       'Borrowing Cancellation Reason', 'Borrowing Request Status',
       'Request Completed Indicator', 'Borrowing Is Filled', 'Borrowing Note', 'User Group Code',
       'To IZ', 'To Library Code', 'To Library Name',
       'Borrowing Creation Date', 'Lending Partner Id', 'Lending Partner Code',
       'Request Type', 'Borrowing Is test']

In [11]:
bor_to_ren_req.columns

Index(['External Identifier', 'Internal Id', 'Shipping Date (Calendar)',
       'Item Loan Date (Calendar)', 'Item Return Date (Calendar)',
       'Item Arrival Date (Calendar)', 'Lending Institution',
       'Cancellation Reason', 'Borrowing Request Status',
       'Request Completed Indicator', 'Is Filled', 'Note', 'User Group Code',
       'Institution Code', 'Library Code', 'Library Name',
       'Borrowing Creation Date', 'Partner Id', 'Partner Code', 'Partner Name',
       'Request Type', 'Title', 'ISSN', 'ISBN', 'MMS Id', 'Borrowing Is test'],
      dtype='object')

In [12]:
# Rename columns
bor_to_ren_req.columns = ['External Identifier',
                          'Borrowing Internal Id',
                          'Item Sent Date',
                          'Item Loan Date',
                          'Item Return Date',
                          'Item Arrival Date',
                          'Lending Institution',
                          'Borrowing Cancellation Reason',
                          'Borrowing Request Status',
                          'Request Completed Indicator',
                          'Borrowing Is Filled',
                          'Borrowing Note',
                          'User Group Code',
                          'To IZ',
                          'To Library Code',
                          'To Library Name',
                          'Borrowing Creation Date',
                          'From IZ',
                          'From Library Code',
                          'From Library Name',
                          'Request Type',
                          'Title',
                          'ISSN',
                          'ISBN',
                          'MMS Id',
                          'Borrowing Is test']

# Move Renouvaud partner to the equivalent of borrowing requests
bor_to_ren_req.loc[bor_to_ren_req['From Library Name'].str.contains('Renouvaud'), 'From IZ'] = 'RENOUVAUD'
bor_to_ren_req['From Library Code'] = bor_to_ren_req['From Library Code'].str.split(' > ').apply(lambda x: x[-1])
bor_to_ren_req['From Library Name'] = bor_to_ren_req['From Library Name'].str.split(' > ').apply(lambda x: x[-1])

In [13]:
# Merge of lending requests with borrowing requests
req = lend_req.merge(bor_req, left_on='Simple external Id', right_on='External Identifier', how='left')

# Data of Renouvaud is incomplete, simply add to the end the remaining data of Renouvaud
req = pd.concat([req, bor_to_ren_req])

In [14]:
req.columns

Index(['External Request Id', 'Lending Internal Id', 'Item Sent Date',
       'Lending Request Status', 'Lending Note', 'From IZ',
       'From Library Code', 'From Library Name',
       'Permanent Physical Location Code', 'Permanent Call Number', 'Barcode',
       'MMS Id', 'Network Id', 'ISSN', 'ISBN', 'Title', 'Borrowing Partner Id',
       'Borrowing Partner Name', 'Lending Is test', 'Simple external Id',
       'External Identifier', 'Borrowing Internal Id', 'Shipping Date',
       'Item Loan Date', 'Item Return Date', 'Item Arrival Date',
       'Lending Institution', 'Borrowing Cancellation Reason',
       'Borrowing Request Status', 'Request Completed Indicator',
       'Borrowing Is Filled', 'Borrowing Note', 'User Group Code', 'To IZ',
       'To Library Code', 'To Library Name', 'Borrowing Creation Date',
       'Lending Partner Id', 'Lending Partner Code', 'Request Type',
       'Borrowing Is test'],
      dtype='object')

In [15]:
# Group test columns
req['Is test'] = req['Lending Is test'] | req['Borrowing Is test'].fillna(False)
req.drop(['Lending Is test', 'Borrowing Is test'], axis=1, inplace=True)

# Move Renouvaud partner to the equivalent of borrowing requests
req.loc[req['Borrowing Partner Name'].fillna('').str.contains('Renouvaud'), 'To IZ'] = 'RENOUVAUD'
req.loc[req['Borrowing Partner Name'].fillna('').str.contains('Renouvaud'), 'To Library Name'] = req.loc[req['Borrowing Partner Name'].fillna('').str.contains('Renouvaud'), 'Borrowing Partner Name'].str.split(' > ').apply(lambda x: x[-1])

# Copy missing column of borrowing requests columns from lending data
def copy_col(row):
    if pd.isnull(row['External Identifier']):
        row['External Identifier'] = row['Simple external Id']
    if pd.isnull(row['External Identifier']):
        row['External Identifier'] = row['Simple external Id']
    if pd.isnull(row['ISBN']):
        row['ISBN'] = row['ISSN']
    if pd.isnull(row['Item Sent Date']):
        row['Item Sent Date'] = row['Shipping Date']
    return row

req = req.apply(copy_col, axis=1)

# Drop some duplicated columns
req.drop(['Simple external Id', 'Lending Partner Id', 'Lending Partner Code', 'Request Completed Indicator', 'ISSN', 'Shipping Date'], axis=1, inplace=True)
    
req = req.rename(columns={'ISBN':'ISBN/ISSN'})    

req.columns

Index(['External Request Id', 'Lending Internal Id', 'Item Sent Date',
       'Lending Request Status', 'Lending Note', 'From IZ',
       'From Library Code', 'From Library Name',
       'Permanent Physical Location Code', 'Permanent Call Number', 'Barcode',
       'MMS Id', 'Network Id', 'ISBN/ISSN', 'Title', 'Borrowing Partner Id',
       'Borrowing Partner Name', 'External Identifier',
       'Borrowing Internal Id', 'Item Loan Date', 'Item Return Date',
       'Item Arrival Date', 'Lending Institution',
       'Borrowing Cancellation Reason', 'Borrowing Request Status',
       'Borrowing Is Filled', 'Borrowing Note', 'User Group Code', 'To IZ',
       'To Library Code', 'To Library Name', 'Borrowing Creation Date',
       'Request Type', 'Is test'],
      dtype='object')

In [16]:
# Reorganise columns
req = req[['External Request Id',
           'External Identifier',
           'Lending Internal Id',
           'Borrowing Internal Id',
           'From IZ',
           'From Library Code',
           'From Library Name',
           'To IZ',
           'To Library Code',
           'To Library Name',
           'Permanent Physical Location Code',
           'Permanent Call Number',
           'Barcode',
           'MMS Id',
           'Network Id', 
           'ISBN/ISSN',
           'Title',
           'Borrowing Creation Date',
           'Item Sent Date',
           'Item Arrival Date',
           'Item Loan Date',
           'Item Return Date',
           'Lending Request Status',
           'Borrowing Request Status',
           'Borrowing Cancellation Reason',
           'Lending Note',
           'Borrowing Note',
           'User Group Code',
           'Borrowing Is Filled',
           'Is test',
           'Lending Institution',
           'Borrowing Partner Name']]

In [17]:
req.to_excel('data/data.xlsx', index=False)

# Build reports
- not test
- TO IZ / From IZ
- ['Borrowing Request Status'] not in ['Lending Request Status', 'Deleted', 'Shipped Digitally', 'Rejected by partner', 'Cancelled by partner']
- ['Lending Request Status'] not in ['Deleted', 'Rejected the borrower request', 'Cancelled']

### Summary report data
- not test
- TO IZ / From IZ must be available
- ['Lending Request Status'] not in ['Deleted', 'Rejected the borrower request', 'Cancelled']

In [18]:
# Suppress requests without IZ info
req_simple = req.loc[~pd.isnull(req['To IZ'])].copy()
req_simple  = req_simple.loc[~pd.isnull(req_simple ['From IZ'])]

# Suppress test requests 
req_simple  = req_simple.loc[~req_simple ['Is test']]

# Suppress cancelled lending requests (lending are master)
req_simple = req_simple.loc[~req_simple ['Lending Request Status'].isin(['Deleted', 'Rejected the borrower request', 'Cancelled'])]

# Keep only IZ information and unique ID
data = req_simple[['From IZ', 'To IZ']].copy()
data['Count'] = 1

# Build the table: belonging IZ on rows and pickup IZ on columns
data = pd.pivot_table(data, columns='To IZ', index='From IZ', aggfunc='count', margins=True).fillna(0).astype(int)
data = data.droplevel(0, axis=1).copy()
data.index.name = 'Owner IZ'
data.columns.name = 'Transit and loan at IZ'

In [19]:
# Rapido_requests_summary
wb = Workbook()
ws = wb.active
ws['A1'] = 'Network circulation summary'
ws['A1'].font = Font(size=16, bold=True)
ws['A2'] = 'This table contains only physical Rapido requests with "sent date" between 01-01-2023 - 30-06-2023 (lending request domain).'
ws['A3'] = 'When destination IZ is Renouvaud, "shipping date" is used (borrowing request domain)'
ws['A4'] = ''
ws['B5'] = 'Destination IZ'
ws['B5'].font = Font(bold=True)

for r in dataframe_to_rows(data, index=True, header=True):
    ws.append(r)

for cell in [i[0] for i in ws[f'A6:A{ws.max_row}']] + list(ws[6]):
    cell.style = 'Pandas'

dim_holder = DimensionHolder(worksheet=ws)
for col in range(ws.min_column, ws.max_column + 1):
    if get_column_letter(col) == 'A':
        dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=10)
    else:
        dim_holder[get_column_letter(col)] = ColumnDimension(ws, min=col, max=col, width=7)
ws.column_dimensions = dim_holder

wb.save(f'Rapido_requests_summary_2023_01-06.xlsx')

### Reports per IZ

In [20]:
# Rapido_requests_owning_inst
df = req.copy()
wb_exp = load_workbook('Explanations.xlsx')
ws_exp = wb_exp.active

for iz in df['From IZ'].unique():
    
    # Filter by owning IZ
    data = df.loc[df['From IZ']==iz]
    
    wb = Workbook()
    ws = wb.active

    for r in dataframe_to_rows(data, index=False, header=True):
        ws.append(r)

    for cell in ws['A'] + ws[1]:
        cell.style = 'Pandas'
    
    ws_2 = wb.create_sheet("Explanations")
    copy_sheet_attributes(ws_exp, ws_2)
    copy_cell_contents(ws_exp, ws_2)
    wb.active = 0
    ws = wb.active
    set_columns_width(ws)
    wb.save(f'reports/{iz}_Rapido_requests_owning_inst_2023_01-06.xlsx')

In [21]:
# Rapido_requests_pickup_loc
for iz in df['To IZ'].unique():
    if pd.isnull(iz):
        continue
        
    # Filter by destination IZ (pickup location)
    data = df.loc[df['To IZ']==iz]
    
    wb = Workbook()
    ws = wb.active

    for r in dataframe_to_rows(data, index=False, header=True):
        ws.append(r)

    for cell in ws['A'] + ws[1]:
        cell.style = 'Pandas'
    
    ws_2 = wb.create_sheet("Explanations")
    copy_sheet_attributes(ws_exp, ws_2)
    copy_cell_contents(ws_exp, ws_2)
    wb.active = 0
    ws = wb.active
    set_columns_width(ws)
    wb.save(f'reports/{iz}_Rapido_requests_pickup_loc_2022_11-12.xlsx')